In [136]:
import numpy as np 
import pandas as pd 
import duckdb as db
conn = db.connect()

Read in main table:

In [137]:
apps = conn.execute(
    """
    DROP TABLE IF EXISTS apps;
    CREATE TABLE apps AS
        SELECT * FROM read_csv_auto('og_data/application_train.csv');
    SELECT * FROM apps;
    """
).fetchdf()

In [138]:
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
apps.TARGET.value_counts()

TARGET
0    282686
1     24825
Name: count, dtype: int64

Very imbalanced, will have to consider oversampling or undersamping.

We noticed many columns are ful of missing values, so we will remove ones with a significant proportion missing. This will be focused on the positive class (minority) to preserve as much information as possible -- there should be enough from the majority class to match however much we end up having in the minority class.

In [140]:
apps = apps.loc[:, apps[apps['TARGET'] == 1].isnull().mean() < 0.5]
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 73 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int64  
 1   TARGET                       307511 non-null  int64  
 2   NAME_CONTRACT_TYPE           307511 non-null  object 
 3   CODE_GENDER                  307511 non-null  object 
 4   FLAG_OWN_CAR                 307511 non-null  object 
 5   FLAG_OWN_REALTY              307511 non-null  object 
 6   CNT_CHILDREN                 307511 non-null  int64  
 7   AMT_INCOME_TOTAL             307511 non-null  float64
 8   AMT_CREDIT                   307511 non-null  float64
 9   AMT_ANNUITY                  307499 non-null  float64
 10  AMT_GOODS_PRICE              307233 non-null  float64
 11  NAME_TYPE_SUITE              306219 non-null  object 
 12  NAME_INCOME_TYPE             307511 non-null  object 
 13 

There are about 8 features left that if we remove them, we will have a very complete dataset. After investigating these specific columns, we decided they are not relevent enough to keep beyond the information we already have. 

In [141]:
apps.drop(columns=['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
                   'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
                   'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
                   'OCCUPATION_TYPE', 'EXT_SOURCE_3'], inplace=True)

In [142]:
apps[apps['TARGET'] == 1].isnull().sum().sum()

287

From the positive class, there are now only 287 total missing values (out of 24k+ rows).

With these missing values, we can either remove the rows or impute. To decide this, we will continue looking at the minority class and seeing if the missing data is concentrated on specific applicants or spread around.

In [143]:
# see if missing data in minority class is concentrated on certain applicatants or spread out
apps['missing_count'] = apps.isnull().sum(axis=1)
apps_minority = apps[apps['TARGET'] == 1]
apps_minority['missing_count'].value_counts().sort_index()

missing_count
0    24667
1      101
2       21
4       36
Name: count, dtype: int64

Only 157 applicants have any missing values. This is pretty negligble compared to the 24,825 applicants in the minority class that we started with. So, I think it's safe to just drop them.

I think we can do the same for the majority class, as long as we still have at least 24k complete observations to balance with the minority class.

In [144]:
apps = apps.dropna()
apps.TARGET.value_counts()

TARGET
0    279864
1     24667
Name: count, dtype: int64

In [145]:
apps.to_csv('data/apps_cleaned.csv', index=False)